# Evaluation of MSSW on AGRAW2

## Imports

In [3]:
import numpy as np
import pandas as pd
import sklearn

## AGRAW2 dataset locations

In [4]:
abrupt_agraw2_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw2_1_abrupt_drift_0_noise_balanced.arff'
gradual_agraw2_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/agraw2_1_gradual_drift_0_noise_balanced_20.arff'
]

all_agraw2_data_paths = [abrupt_agraw2_path] + gradual_agraw2_paths

## Accept and preprocess AGRAW2 datasets

In [5]:
import accepting
from sklearn.preprocessing import MinMaxScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split


agraw2_exclude_reference_batches = {}
agraw2_exclude_testing_batches = {}
agraw2_onehot_reference_batches = {}
agraw2_onehot_testing_batches = {}
agraw2_target_reference_batches = {}
agraw2_target_testing_batches = {}


# agraw2 with categories excluded
for file_path in all_agraw2_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    reference_data = df_x_ref_num.to_numpy()
    testing_data = df_x_test_num.to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw2_exclude_reference_batches[file_path] = ref_batches
    agraw2_exclude_testing_batches[file_path] = test_batches
    
print('agraw2 exclude')
print(agraw2_exclude_reference_batches)
print(agraw2_exclude_testing_batches)

# agraw2 with categories onehot encoded
for file_path in all_agraw2_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(df_x_ref_cat)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw2_onehot_reference_batches[file_path] = ref_batches
    agraw2_onehot_testing_batches[file_path] = test_batches
    
print('agraw2 onehot')
print(agraw2_onehot_reference_batches)
print(agraw2_onehot_testing_batches)

# agraw2 with categories target encoded
for file_path in all_agraw2_data_paths:
    df_x, df_y = accepting.get_clean_df(file_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    df_x_ref_num, df_x_ref_cat = accepting.divide_numeric_categorical(df_x_ref)
    df_x_test_num, df_x_test_cat = accepting.divide_numeric_categorical(df_x_test)
    
    ref_index = df_x_ref_cat.index
    test_index = df_x_test_cat.index
    encoder = TargetEncoder()
    encoder.fit(df_x_ref_cat, df_y_ref)
    df_x_ref_cat_transformed = pd.DataFrame(encoder.transform(df_x_ref_cat))
    df_x_test_cat_transformed = pd.DataFrame(encoder.transform(df_x_test_cat))
    df_x_ref_cat_transformed.set_index(ref_index, inplace=True)
    df_x_test_cat_transformed.set_index(test_index, inplace=True)
    
    reference_data = df_x_ref_num.join(df_x_ref_cat_transformed, lsuffix='_num').to_numpy()
    testing_data = df_x_test_num.join(df_x_test_cat_transformed, lsuffix='_num').to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    agraw2_target_reference_batches[file_path] = ref_batches
    agraw2_target_testing_batches[file_path] = test_batches
    
print('agraw2 target')
print(agraw2_target_reference_batches)
print(agraw2_target_testing_batches)

df               salary    commission   age     elevel       car      zipcode  \
0       58227.414161  42921.435776  77.0  b'level2'   b'car6'  b'zipcode5'   
1       55628.982351  22363.143593  27.0  b'level3'  b'car17'  b'zipcode3'   
2      146598.421289      0.000000  26.0  b'level2'  b'car13'  b'zipcode5'   
3       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
4       83888.656723      0.000000  72.0  b'level3'  b'car11'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995  138438.140611      0.000000  50.0  b'level2'   b'car8'  b'zipcode6'   
99996   90804.449088      0.000000  77.0  b'level4'  b'car13'  b'zipcode7'   
99997  106569.365368      0.000000  51.0  b'level1'   b'car8'  b'zipcode6'   
99998   84356.498251      0.000000  36.0  b'level1'  b'car18'  b'zipcode5'   
99999   87473.968095      0.000000  54.0  b'level0'  b'car11'  b'zipcode3'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df               salary    commission   age     elevel       car      zipcode  \
0       58227.414161  42921.435776  77.0  b'level2'   b'car6'  b'zipcode5'   
1       55628.982351  22363.143593  27.0  b'level3'  b'car17'  b'zipcode3'   
2      146598.421289      0.000000  26.0  b'level2'  b'car13'  b'zipcode5'   
3       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
4       83888.656723      0.000000  72.0  b'level3'  b'car11'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   39396.704989  18582.273595  72.0  b'level1'  b'car16'  b'zipcode5'   
99996  138438.140611      0.000000  50.0  b'level2'   b'car8'  b'zipcode6'   
99997   90804.449088      0.000000  77.0  b'level4'  b'car13'  b'zipcode7'   
99998  106569.365368      0.000000  51.0  b'level1'   b'car8'  b'zipcode6'   
99999   84356.498251      0.000000  36.0  b'level1'  b'car18'  b'zipcode5'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df               salary    commission   age     elevel       car      zipcode  \
0       58227.414161  42921.435776  77.0  b'level2'   b'car6'  b'zipcode5'   
1       55628.982351  22363.143593  27.0  b'level3'  b'car17'  b'zipcode3'   
2      146598.421289      0.000000  26.0  b'level2'  b'car13'  b'zipcode5'   
3       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
4       83888.656723      0.000000  72.0  b'level3'  b'car11'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995  111359.652359      0.000000  61.0  b'level1'  b'car15'  b'zipcode1'   
99996  147593.942041      0.000000  59.0  b'level4'  b'car17'  b'zipcode6'   
99997  123948.303642      0.000000  66.0  b'level4'  b'car11'  b'zipcode4'   
99998   71333.696488  11799.103316  58.0  b'level1'   b'car5'  b'zipcode3'   
99999   39396.704989  18582.273595  72.0  b'level1'  b'car16'  b'zipcode5'   

              hvalue  hyears           loan      class  
0  

...

df               salary    commission   age     elevel       car      zipcode  \
0       58227.414161  42921.435776  77.0  b'level2'   b'car6'  b'zipcode5'   
1       55628.982351  22363.143593  27.0  b'level3'  b'car17'  b'zipcode3'   
2      146598.421289      0.000000  26.0  b'level2'  b'car13'  b'zipcode5'   
3       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
4       83888.656723      0.000000  72.0  b'level3'  b'car11'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   68222.567612  33141.251698  41.0  b'level2'   b'car8'  b'zipcode1'   
99996   50428.438359  59324.405616  26.0  b'level4'   b'car1'  b'zipcode2'   
99997  112914.978798      0.000000  41.0  b'level1'   b'car4'  b'zipcode4'   
99998   90178.866228      0.000000  60.0  b'level1'  b'car13'  b'zipcode6'   
99999   37082.251910  22468.199196  43.0  b'level2'  b'car20'  b'zipcode1'   

             hvalue  hyears           loan      class  
0   

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warni

df               salary    commission   age     elevel       car      zipcode  \
0       58227.414161  42921.435776  77.0  b'level2'   b'car6'  b'zipcode5'   
1       55628.982351  22363.143593  27.0  b'level3'  b'car17'  b'zipcode3'   
2      146598.421289      0.000000  26.0  b'level2'  b'car13'  b'zipcode5'   
3       87701.290931      0.000000  52.0  b'level4'  b'car16'  b'zipcode5'   
4       83888.656723      0.000000  72.0  b'level3'  b'car11'  b'zipcode5'   
...              ...           ...   ...        ...       ...          ...   
99995   80507.066152      0.000000  59.0  b'level1'   b'car8'  b'zipcode7'   
99996   43953.844702  23112.195013  36.0  b'level1'  b'car19'  b'zipcode6'   
99997  127958.679326      0.000000  59.0  b'level2'   b'car8'  b'zipcode7'   
99998   58150.729498  54772.811468  66.0  b'level3'  b'car15'  b'zipcode8'   
99999  120616.756315      0.000000  59.0  b'level2'   b'car3'  b'zipcode3'   

              hvalue  hyears           loan      class  
0  

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warni

agraw2 target
{'../Datasets_concept_drift/synthetic_data/abrupt_drift/agraw2_1_abrupt_drift_0_noise_balanced.arff': [array([[5.82274142e+04, 4.29214358e+04, 7.70000000e+01, ...,
        5.00332226e-01, 4.81505516e-01, 5.08792497e-01],
       [5.56289824e+04, 2.23631436e+04, 2.70000000e+01, ...,
        4.96433903e-01, 5.04279131e-01, 5.12541553e-01],
       [1.46598421e+05, 0.00000000e+00, 2.60000000e+01, ...,
        5.00332226e-01, 4.92203390e-01, 5.08792497e-01],
       ...,
       [2.48369729e+04, 7.16794719e+04, 3.90000000e+01, ...,
        5.03981027e-01, 5.04279131e-01, 5.12541553e-01],
       [6.03088864e+04, 6.70882404e+04, 5.20000000e+01, ...,
        5.03981027e-01, 4.95765472e-01, 5.06815366e-01],
       [6.50239030e+04, 4.07706319e+04, 5.90000000e+01, ...,
        4.96433903e-01, 4.95675316e-01, 4.97927768e-01]]), array([[1.28572172e+05, 0.00000000e+00, 6.30000000e+01, ...,
        4.99582847e-01, 5.07061197e-01, 4.81747457e-01],
       [3.28746914e+04, 4.94224358e+04, 4.9

## AGRAW2 with categories excluded

### Find the best tol and max_iter (the drift type is irrelevant)

In [12]:
import mssw_preprocessing
import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    agraw2_exclude_reference_batches[abrupt_agraw2_path], agraw2_exclude_testing_batches[abrupt_agraw2_path])

filename = 'agraw2_exclude_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 27
minimum initial inertia: 773.212821848988
maximum initial inertia: 1541.4774421685145
number of unique final inertia values: 5
minimum final inertia: 593.2816577731598
maximum final inertia: 593.281665957085
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


### Use them for the analysis

In [13]:
import mssw_eval


agraw2_exclude_stats = {}
for agraw2_path in all_agraw2_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        agraw2_exclude_reference_batches[agraw2_path],
        agraw2_exclude_testing_batches[agraw2_path],
        n_clusters=2,
        n_init=100,
        max_iter=300,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw2_exclude_stats[agraw2_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('agraw2 STATS')
print(agraw2_exclude_stats)

min_runs 2
centroid distance sums [[1324.80830567 1045.56993291]]
num points in clusters [[5756. 4244.]]
centroid distance sums [[1299.04190413 1076.82258213]]
num points in clusters [[5630. 4370.]]
centroid distance sums [[1332.92632246 1042.54999585]]
num points in clusters [[5783. 4217.]]
centroid distance sums [[1337.6280429  1027.11298171]]
num points in clusters [[5826. 4174.]]
centroid distance sums [[1304.14040555 1074.23665677]]
num points in clusters [[5646. 4354.]]
centroid distance sums [[1324.6414027  1013.16999339]]
num points in clusters [[5831. 4169.]]
centroid distance sums [[1313.41676921 1010.3838153 ]]
num points in clusters [[5810. 4190.]]
centroid distance sums [[1296.31358461 1016.82876411]]
num points in clusters [[5766. 4234.]]
centroid distance sums [[1327.64394339  983.77203327]]
num points in clusters [[5914. 4086.]]
centroid distance sums [[1320.04064975 1004.01590568]]
num points in clusters [[5832. 4168.]]
centroid distance sums [[1045.56993291 1324.80830

centroid distance sums [[1045.56993291 1324.80830567]]
num points in clusters [[4244. 5756.]]
centroid distance sums [[1076.82258213 1299.04190413]]
num points in clusters [[4370. 5630.]]
centroid distance sums [[1042.54999585 1332.92632246]]
num points in clusters [[4217. 5783.]]
centroid distance sums [[1027.39446251 1337.44533374]]
num points in clusters [[4175. 5825.]]
centroid distance sums [[1072.06291563 1305.18692381]]
num points in clusters [[4346. 5654.]]
centroid distance sums [[1037.38866971 1322.64611178]]
num points in clusters [[4232. 5768.]]
centroid distance sums [[1008.71287796 1319.39489572]]
num points in clusters [[4167. 5833.]]
centroid distance sums [[1021.74542297 1299.03083761]]
num points in clusters [[4246. 5754.]]
centroid distance sums [[1009.06596419 1304.10266892]]
num points in clusters [[4176. 5824.]]
centroid distance sums [[ 977.25520321 1337.5274978 ]]
num points in clusters [[4089. 5911.]]
min_runs 2
centroid distance sums [[1324.80830567 1045.56993

### Save the results

In [14]:
import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw2_exclude_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'exclude'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(300)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw2_exclude_jupyter.csv', index=False)

## AGRAW2 with categories onehot encoded

### Find the best tol and max_iter (the drift type is irrelevant)

In [15]:
import mssw_preprocessing
import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    agraw2_onehot_reference_batches[abrupt_agraw2_path], agraw2_onehot_testing_batches[abrupt_agraw2_path])

filename = 'agraw2_onehot_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 29
minimum initial inertia: 1314.4539431527146
maximum initial inertia: 1437.617718127152
number of unique final inertia values: 20
minimum final inertia: 730.5627872414303
maximum final inertia: 750.120304150691
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


### Use them for the analysis

In [16]:
import mssw_eval


agraw2_onehot_stats = {}
for agraw2_path in all_agraw2_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        agraw2_onehot_reference_batches[agraw2_path],
        agraw2_onehot_testing_batches[agraw2_path],
        n_clusters=2,
        n_init=100,
        max_iter=300,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw2_onehot_stats[agraw2_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('agraw2 STATS')
print(agraw2_onehot_stats)

min_runs 2
centroid distance sums [[2191.69834903  505.4337576 ]]
num points in clusters [[7903. 2097.]]
centroid distance sums [[2236.18120208  468.66118102]]
num points in clusters [[8057. 1943.]]
centroid distance sums [[2214.61543224  486.58407273]]
num points in clusters [[7985. 2015.]]
centroid distance sums [[2231.70838812  470.81064385]]
num points in clusters [[8048. 1952.]]
centroid distance sums [[2224.43738848  478.10660645]]
num points in clusters [[8019. 1981.]]
centroid distance sums [[2217.47106673  480.64021873]]
num points in clusters [[8003. 1997.]]
centroid distance sums [[2211.01305538  485.48835851]]
num points in clusters [[7983. 2017.]]
centroid distance sums [[2220.90674611  476.07776756]]
num points in clusters [[8023. 1977.]]
centroid distance sums [[2230.7476847   466.92550987]]
num points in clusters [[8056. 1944.]]
centroid distance sums [[2238.66553551  461.2779717 ]]
num points in clusters [[8081. 1919.]]
centroid distance sums [[ 505.4337576  2191.69834

centroid distance sums [[ 505.4337576  2191.69834903]]
num points in clusters [[2097. 7903.]]
centroid distance sums [[ 468.66118102 2236.18120208]]
num points in clusters [[1943. 8057.]]
centroid distance sums [[ 486.58407273 2214.61543224]]
num points in clusters [[2015. 7985.]]
centroid distance sums [[ 470.81064385 2231.71275616]]
num points in clusters [[1952. 8048.]]
centroid distance sums [[ 478.08840379 2224.29821691]]
num points in clusters [[1981. 8019.]]
centroid distance sums [[ 475.2035692  2225.78100759]]
num points in clusters [[1973. 8027.]]
centroid distance sums [[ 492.21174175 2203.93064689]]
num points in clusters [[2043. 7957.]]
centroid distance sums [[ 474.72911046 2222.96854017]]
num points in clusters [[1974. 8026.]]
centroid distance sums [[ 471.84707219 2225.86610061]]
num points in clusters [[1961. 8039.]]
centroid distance sums [[ 469.6556221 2228.1038424]]
num points in clusters [[1954. 8046.]]
min_runs 2
centroid distance sums [[2191.69834903  505.4337576

### Save the results

In [18]:
import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw2_onehot_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'onehot'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(300)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw2_onehot_jupyter.csv', index=False)

## AGRAW2 with categories target encoded

### Find the best tol and max_iter (the drift type is irrelevant)

In [19]:
import mssw_preprocessing
import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    agraw2_target_reference_batches[abrupt_agraw2_path], agraw2_target_testing_batches[abrupt_agraw2_path])

filename = 'agraw2_target_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 21
minimum initial inertia: 888.8466859654951
maximum initial inertia: 1776.891247174513
number of unique final inertia values: 6
minimum final inertia: 685.5043938637058
maximum final inertia: 760.4975209431744
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


### Use them for the analysis

In [20]:
import mssw_eval


agraw2_target_stats = {}
for agraw2_path in all_agraw2_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        agraw2_target_reference_batches[agraw2_path],
        agraw2_target_testing_batches[agraw2_path],
        n_clusters=2,
        n_init=100,
        max_iter=250,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    agraw2_target_stats[agraw2_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('agraw2 STATS')
print(agraw2_target_stats)

min_runs 2
centroid distance sums [[1447.49474776 1116.14855134]]
num points in clusters [[5754. 4246.]]
centroid distance sums [[1425.49318419 1149.02686135]]
num points in clusters [[5632. 4368.]]
centroid distance sums [[1460.985225   1106.47708818]]
num points in clusters [[5783. 4217.]]
centroid distance sums [[1473.35976358 1099.75100553]]
num points in clusters [[5823. 4177.]]
centroid distance sums [[1435.07683722 1140.85684185]]
num points in clusters [[5644. 4356.]]
centroid distance sums [[1463.7476838  1087.40718078]]
num points in clusters [[5830. 4170.]]
centroid distance sums [[1454.88498561 1091.43617821]]
num points in clusters [[5811. 4189.]]
centroid distance sums [[1433.70186962 1097.01755707]]
num points in clusters [[5764. 4236.]]
centroid distance sums [[1473.6534033 1064.5231286]]
num points in clusters [[5911. 4089.]]
centroid distance sums [[1467.18653677 1077.25777837]]
num points in clusters [[5836. 4164.]]
centroid distance sums [[1116.14855134 1447.4947477

centroid distance sums [[1116.14855134 1447.49474776]]
num points in clusters [[4246. 5754.]]
centroid distance sums [[1149.02686135 1425.49318419]]
num points in clusters [[4368. 5632.]]
centroid distance sums [[1106.47708818 1460.985225  ]]
num points in clusters [[4217. 5783.]]
centroid distance sums [[1099.99225988 1473.25925589]]
num points in clusters [[4178. 5822.]]
centroid distance sums [[1138.60310641 1436.91450089]]
num points in clusters [[4348. 5652.]]
centroid distance sums [[1109.55792947 1457.40068068]]
num points in clusters [[4233. 5767.]]
centroid distance sums [[1085.31214859 1457.4400134 ]]
num points in clusters [[4168. 5832.]]
centroid distance sums [[1101.25626658 1437.95070869]]
num points in clusters [[4245. 5755.]]
centroid distance sums [[1088.11839369 1446.28813827]]
num points in clusters [[4178. 5822.]]
centroid distance sums [[1058.1411808  1487.57460156]]
num points in clusters [[4088. 5912.]]
min_runs 2
centroid distance sums [[1447.52579432 1116.60017

### Save the results

In [29]:
import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in agraw2_target_stats.items():
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'target'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(300)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('agraw2_target_jupyter.csv', index=False)